<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/Visualizacion_datos/blob/main/Pr2_ProcesamientoVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install opencv_jupyter_ui
!pip install -q ipycanvas==0.11
from google.colab import output
output.enable_custom_widget_manager()
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.7/255.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.3 MB/s eta 0:00:00


In [ ]:
import cv2
from matplotlib import pyplot as plt
import opencv_jupyter_ui as jcv2

In [ ]:
from google.colab import files
uploadfile=files.upload()

Saving Mall_Security_Cam.mp4 to Mall_Security_Cam.mp4


In [ ]:
from google.colab.patches import cv2_imshow
# Video
cap = cv2.VideoCapture("Mall_Security_Cam.mp4")

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")


ret, frame1 = cap.read(1)
ret, frame2 = cap.read(2)

while(cap.isOpened()):
  diff = cv2.absdiff(frame1,frame2)
  gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (5,5), 0)
  _, mask = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
  dil = cv2.dilate(mask, None, iterations=5)
  cont, _ = cv2.findContours(dil, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  cv2.drawContours(frame1, cont, -1, (0, 255, 0), 2)

  jcv2.imshow(, frame1)
  frame1 = frame2
  ret, frame2 = cap.read()

  if cv2.waitKey(3) == 5:
    break

cv2.destroyAllWindows()
cap.release()

error: ignored